In [1]:
using Symbolics

In [2]:
@variables x y C₁ C₂ C₃;

In [3]:
Dx = Differential(x)
Dy = Differential(y);

In [4]:
ϕ = C₁ * x^2 + C₂ * x * y + C₃ * y^2

C₁*(x^2) + C₂*x*y + C₃*(y^2)

In [5]:
σₓ = expand_derivatives(Dx(Dx(ϕ)))

2C₁

In [6]:
using LowLevelFEM
import LowLevelFEM as FEM

gmsh.initialize()

Info    : Increasing process stack size (8192 kB < 16 MB)


In [7]:
gmsh.open("beam.geo")

Info    : Reading 'beam.geo'...
Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 30%] Meshing curve 2 (Line)
Info    : [ 60%] Meshing curve 3 (Line)
Info    : [ 80%] Meshing curve 4 (Line)
Info    : Done meshing 1D (Wall 0.00076522s, CPU 0.000743s)
Info    : Meshing 2D...
Info    : Meshing surface 1 (Plane, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 0.0692035s, CPU 0.066452s)
Info    : 1314 nodes 2630 elements
Info    : Done reading 'beam.geo'


In [8]:
#gmsh.fltk.run()

In [9]:
mat = FEM.material("body")
problem = FEM.Problem([mat], type=:PlaneStress);

Info    : RCMK renumbering...
Info    : Done RCMK renumbering (bandwidth is now 18)


In [10]:
M = 1
l = 100
h = 10
M0(x, y, z) = 12M / h^3 * (y - h / 2)
load = FEM.load("right", fx=M0)
supp = FEM.displacementConstraint("left", ux=0, uy=0);

In [11]:
q = FEM.solveDisplacement(problem, [load], [supp]);

In [12]:
S = FEM.solveStress(problem, q)
S = FEM.elementsToNodes(problem, S);

In [13]:
sx = FEM.showDoFResults(problem, S, :sx)

-------------------------------------------------------
Version       : 4.13.1
License       : GNU General Public License
Build OS      : Linux64-sdk
Build date    : 19700101
Build host    : amdci7.julia.csail.mit.edu
Build options : 64Bit ALGLIB[contrib] ANN[contrib] Bamg Blossom Cairo DIntegration Dlopen DomHex Eigen[contrib] Fltk GMP Gmm[contrib] Hxt Jpeg Kbipack LinuxJoystick MathEx[contrib] Mesh Metis[contrib] Mmg Mpeg Netgen Nii2mesh ONELAB ONELABMetamodel OpenCASCADE OpenCASCADE-CAF OpenGL OpenMP OptHom Parser Plugins Png Post QuadMeshingTools QuadTri Solver TetGen/BR TinyXML2[contrib] Untangle Voro++[contrib] WinslowUntangler Zlib
FLTK version  : 1.3.8
OCC version   : 7.7.2
Packaged by   : root
Web site      : https://gmsh.info
Issue tracker : https://gitlab.onelab.info/gmsh/gmsh/issues
-------------------------------------------------------


XOpenIM() failed
XRequest.18: BadValue 0x0


1

In [14]:
exact(x, y, z) = 12M / h^3 * (y - h / 2)

exact (generic function with 1 method)

In [15]:
sx0 = FEM.displacementConstraint("body", ux=exact)
dof = length(q)
ssx = zeros(dof)
FEM.applyBoundaryConditions!(problem, ssx, [sx0])

In [ ]:
sx1 = FEM.showDoFResults(problem, ssx, :ux)

In [ ]:
gmsh.fltk.run()

In [ ]:
gmsh.finalize()